In [102]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [103]:
## Load the trained model,scaler pikel,onehot

model=load_model('model.h5')

##Load the encoder and scaler

with open('one_hot_encoder.pkl','rb') as file:
    one_hot_encoder=pickle.load(file)
    
    
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)
    

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [104]:
# Check what it contains
print("Loaded encoder classes:", label_encoder_gender.classes_)

Loaded encoder classes: ['Female' 'Male']


In [105]:
# If classes are wrong (like [0 1]), recreate the encoder
if not all(isinstance(cls, str) for cls in label_encoder_gender.classes_):
    print("Encoder was trained on wrong data type - recreating...")
    label_encoder_gender = LabelEncoder()
    label_encoder_gender.fit(['Male', 'Female'])

In [106]:
   # Save the fixed version
with open('label_encoder_gender.pkl', 'wb') as f:
        pickle.dump(label_encoder_gender, f)

In [107]:
## Example data

input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumofProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [108]:
## One-hot encoded " Geography"

geo_encoded =one_hot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [109]:
input_df=pd.DataFrame([input_data])

In [110]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [111]:
## Concatination one hot encoded data

input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [112]:
# Rename the column to match what the scaler expects
input_df = input_df.rename(columns={'NumofProducts': 'NumOfProducts'})

# Now scaling will work
input_scaled = scaler.transform(input_df)

In [113]:
## Predict Churn

prediction=model.predict(input_scaled)

prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step


array([[0.02534798]], dtype=float32)

In [115]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.02534798)

In [116]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
